## <div align="center"> SIMULASI MULTI CRITERIA DECISION MAKING : ELECTRE </div>
## <div align="center"> LAPORAN AKHIR IF541 - EXPERT SYSTEM </div>
### <div align="center"> GROUP B </div>


<h2 style="text-align:center"><i>"Sistem Pemilihan Beasiswa Bantuan Universitas dengan ELECTRE"</i></h2>

<h2>Anggota Kelompok :</h2>
<ul>
    <li>Farrel Dinarta / 00000055702</li>
    <li>Farrel Dinarta / 00000055702</li>
    <li>Farrel Dinarta / 00000055702</li>
    <li>Farrel Dinarta / 00000055702</li>    
<ul>


### Import Libraries

In [375]:
import numpy as np
import pandas as pd
import math

### Import Candidate Data

In [376]:
try:
    data = pd.read_csv("candidate_data.csv")
except FileNotFoundError as e:
    print("File not found.")

### Preprocessing
- First row is reserved specifically for criteria weight
- First row will be removed after separating it from the original dataset
- Second row is reserved specifically to determine whether a label is `benefit` or `cost` attribute.
- Second row will be removed after separating it from the original dataset.

In [377]:
criteria_weights = data.iloc[0]
criteria_types = data.iloc[1]
data = data.iloc[2:]

### Get Criterias

In [378]:
data.columns

Index(['ipk', 'pendapatan_orang_tua', 'biaya_hidup', 'prestasi', 'keaktifan'], dtype='object')

### Get Criteria Types

In [379]:
types = np.unique(criteria_types)
if np.any(types == np.array(['b', 'c'])):
    print(criteria_types)
else:
    raise ValueError("Invalid criteria type.")

ipk                     b
pendapatan_orang_tua    c
biaya_hidup             b
prestasi                b
keaktifan               b
Name: 1, dtype: object


### Get Criteria Weights

In [380]:
# scale the weights to span only from 0 - 1
criteria_weights = {key: float(value) for key, value in criteria_weights.items()}
total_weight = sum([value for key, value in criteria_weights.items()])
criteria_weights = {key : value/total_weight for key, value in criteria_weights.items()}
array_criteria_weights = [value for key, value in criteria_weights.items()]

criteria_weights

{'ipk': 0.3,
 'pendapatan_orang_tua': 0.3,
 'biaya_hidup': 0.15,
 'prestasi': 0.15,
 'keaktifan': 0.1}

### Data Preview

In [381]:
data

,ipk,pendapatan_orang_tua,biaya_hidup,prestasi,keaktifan
2,4.00,2000000,1500000,6,5
3,4.00,4000000,3000000,3,3
4,3.50,8000000,6000000,4,2
5,3.66,1000000,1000000,5,1
6,3.80,2500000,1000000,6,4
7,3.70,5000000,3000000,3,5
8,3.40,1000000,800000,2,5
9,3.20,3500000,3000000,1,4


### Get Criteria and Alternative Counts

In [382]:
shape = data.shape
print(str(shape[0]) + " alternatives")
print(str(shape[1]) + " criterias")

8 alternatives
5 criterias


### STEP 1 : Normalization

In [383]:
def root_of_square_sum(column):
    return math.sqrt(sum(x**2 for x in data[column]))

def normalize(values, column):
    return values / root_of_square_sum(column)

In [384]:
data = data.apply(pd.to_numeric, errors='coerce')

# normalized data 
for column in data.columns:
    data[column] = normalize(data[column], column)
    
data

,ipk,pendapatan_orang_tua,biaya_hidup,prestasi,keaktifan
2,0.385659,0.175750,0.182049,0.514496,0.454545
3,0.385659,0.351500,0.364098,0.257248,0.272727
4,0.337451,0.703000,0.728196,0.342997,0.181818
5,0.352878,0.087875,0.121366,0.428746,0.090909
6,0.366376,0.219687,0.121366,0.514496,0.363636
7,0.356734,0.439375,0.364098,0.257248,0.454545
8,0.327810,0.087875,0.097093,0.171499,0.454545
9,0.308527,0.307562,0.364098,0.085749,0.363636


### STEP 2 : Weighting

In [385]:
def weight(values, column):
    return values * criteria_weights[column]

In [386]:
for column in data.columns:
    data[column] = weight(data[column], column)
    
data

,ipk,pendapatan_orang_tua,biaya_hidup,prestasi,keaktifan
2,0.115698,0.052725,0.027307,0.077174,0.045455
3,0.115698,0.105450,0.054615,0.038587,0.027273
4,0.101235,0.210900,0.109229,0.051450,0.018182
5,0.105863,0.026362,0.018205,0.064312,0.009091
6,0.109913,0.065906,0.018205,0.077174,0.036364
7,0.107020,0.131812,0.054615,0.038587,0.045455
8,0.098343,0.026362,0.014564,0.025725,0.045455
9,0.092558,0.092269,0.054615,0.012862,0.036364


### STEP 3 : Generate Concordance Matrix

In [387]:
def generate_concordance_matrix(data):
    shape = data.shape[0]
    matrix = np.zeros((shape, shape))
    
    concordance_sets = {}
    discordance_sets = {}
    
    for index_i, row_i in data.iterrows():
        for index_j, row_j in data.iterrows():
            if index_i == index_j:
                continue
            cell_set_total = 0
            cset = []            
            dset = []
            for value in range(len(row_i)):
                if ((criteria_types[value] == 'b' and row_i[value] >= row_j[value]) | (criteria_types[value] == 'c' and row_i[value] < row_j[value])):
                    cell_set_total += array_criteria_weights[value]
                    cset.append(value)
                else:
                    dset.append(value)
            concordance_sets[f"C-{index_i-1}{index_j-1}"] = cset
            discordance_sets[f"D-{index_i-1}{index_j-1}"] = dset
            matrix[index_i-2][index_j-2] = cell_set_total        
    return matrix, concordance_sets, discordance_sets

In [388]:
concordance_matrix, concordance_sets, discordance_sets = generate_concordance_matrix(data)
concordance_matrix

array([[0.  , 0.85, 0.85, 0.7 , 1.  , 0.85, 0.7 , 0.85],
       [0.45, 0.  , 0.7 , 0.55, 0.45, 0.9 , 0.6 , 0.6 ],
       [0.15, 0.3 , 0.  , 0.25, 0.15, 0.3 , 0.6 , 0.6 ],
       [0.3 , 0.45, 0.75, 0.  , 0.45, 0.45, 0.6 , 0.75],
       [0.15, 0.55, 0.85, 0.7 , 0.  , 0.75, 0.6 , 0.85],
       [0.25, 0.4 , 0.7 , 0.55, 0.25, 0.  , 0.7 , 0.7 ],
       [0.4 , 0.4 , 0.4 , 0.1 , 0.4 , 0.4 , 0.  , 0.85],
       [0.15, 0.55, 0.4 , 0.25, 0.25, 0.45, 0.15, 0.  ]])

### STEP 4 : Generate Discordance Matrix

- Concordance Sets

In [389]:
concordance_sets

{'C-12': [0, 1, 3, 4],
 'C-13': [0, 1, 3, 4],
 'C-14': [0, 2, 3, 4],
 'C-15': [0, 1, 2, 3, 4],
 'C-16': [0, 1, 3, 4],
 'C-17': [0, 2, 3, 4],
 'C-18': [0, 1, 3, 4],
 'C-21': [0, 2],
 'C-23': [0, 1, 4],
 'C-24': [0, 2, 4],
 'C-25': [0, 2],
 'C-26': [0, 1, 2, 3],
 'C-27': [0, 2, 3],
 'C-28': [0, 2, 3],
 'C-31': [2],
 'C-32': [2, 3],
 'C-34': [2, 4],
 'C-35': [2],
 'C-36': [2, 3],
 'C-37': [0, 2, 3],
 'C-38': [0, 2, 3],
 'C-41': [1],
 'C-42': [1, 3],
 'C-43': [0, 1, 3],
 'C-45': [1, 2],
 'C-46': [1, 3],
 'C-47': [0, 2, 3],
 'C-48': [0, 1, 3],
 'C-51': [3],
 'C-52': [1, 3, 4],
 'C-53': [0, 1, 3, 4],
 'C-54': [0, 2, 3, 4],
 'C-56': [0, 1, 3],
 'C-57': [0, 2, 3],
 'C-58': [0, 1, 3, 4],
 'C-61': [2, 4],
 'C-62': [2, 3, 4],
 'C-63': [0, 1, 4],
 'C-64': [0, 2, 4],
 'C-65': [2, 4],
 'C-67': [0, 2, 3, 4],
 'C-68': [0, 2, 3, 4],
 'C-71': [1, 4],
 'C-72': [1, 4],
 'C-73': [1, 4],
 'C-74': [4],
 'C-75': [1, 4],
 'C-76': [1, 4],
 'C-78': [0, 1, 3, 4],
 'C-81': [2],
 'C-82': [1, 2, 4],
 'C-83': [1, 4],

In [390]:
discordance_sets

{'D-12': [2],
 'D-13': [2],
 'D-14': [1],
 'D-15': [],
 'D-16': [2],
 'D-17': [1],
 'D-18': [2],
 'D-21': [1, 3, 4],
 'D-23': [2, 3],
 'D-24': [1, 3],
 'D-25': [1, 3, 4],
 'D-26': [4],
 'D-27': [1, 4],
 'D-28': [1, 4],
 'D-31': [0, 1, 3, 4],
 'D-32': [0, 1, 4],
 'D-34': [0, 1, 3],
 'D-35': [0, 1, 3, 4],
 'D-36': [0, 1, 4],
 'D-37': [1, 4],
 'D-38': [1, 4],
 'D-41': [0, 2, 3, 4],
 'D-42': [0, 2, 4],
 'D-43': [2, 4],
 'D-45': [0, 3, 4],
 'D-46': [0, 2, 4],
 'D-47': [1, 4],
 'D-48': [2, 4],
 'D-51': [0, 1, 2, 4],
 'D-52': [0, 2],
 'D-53': [2],
 'D-54': [1],
 'D-56': [2, 4],
 'D-57': [1, 4],
 'D-58': [2],
 'D-61': [0, 1, 3],
 'D-62': [0, 1],
 'D-63': [2, 3],
 'D-64': [1, 3],
 'D-65': [0, 1, 3],
 'D-67': [1],
 'D-68': [1],
 'D-71': [0, 2, 3],
 'D-72': [0, 2, 3],
 'D-73': [0, 2, 3],
 'D-74': [0, 1, 2, 3],
 'D-75': [0, 2, 3],
 'D-76': [0, 2, 3],
 'D-78': [2],
 'D-81': [0, 1, 3, 4],
 'D-82': [0, 3],
 'D-83': [0, 2, 3],
 'D-84': [0, 1, 3],
 'D-85': [0, 1, 3],
 'D-86': [0, 3, 4],
 'D-87': [0, 1,

In [391]:
def generate_discordance_matrix(data):
    shape = data.shape[0]
    matrix = np.zeros((shape, shape))
    numpy_data_values = data.values
    
    for index_i, row_i in data.iterrows():
        for index_j, row_j in data.iterrows():
            if index_i == index_j:
                continue
            cell_value = 0
            nominator = [
                abs(numpy_data_values[index_i - 2][idx] - numpy_data_values[index_j - 2][idx])
                for idx in discordance_sets[f"D-{index_i-1}{index_j-1}"]
            ]

            denominator = [
                abs(numpy_data_values[index_i-2][idx] - numpy_data_values[index_j-2][idx])
                for idx in range(numpy_data_values.shape[1])
            ]
                
            if (len(nominator) == 0):
                matrix[index_i-2][index_j-2] = 0
            else:
                matrix[index_i-2][index_j-2] = max(nominator)/max(denominator)
            
    return matrix               

In [392]:
discordance_matrix = generate_discordance_matrix(data)
discordance_matrix 

array([[0.        , 0.51792067, 0.51792067, 0.72496838, 0.        ,
        0.34528045, 0.51239464, 0.42460764],
       [1.        , 0.        , 0.51792067, 1.        , 1.        ,
        0.68968525, 1.        , 0.51239464],
       [1.        , 1.        , 0.        , 1.        , 1.        ,
        1.        , 1.        , 1.        ],
       [1.        , 0.46037393, 0.49325779, 0.        , 0.68968525,
        0.34528045, 0.94237606, 0.55244872],
       [1.        , 0.92074787, 0.62778264, 1.        , 0.        ,
        0.55244872, 0.76859195, 0.56614352],
       [1.        , 1.        , 0.6905609 , 1.        , 1.        ,
        0.        , 1.        , 1.        ],
       [1.        , 0.50641133, 0.5129881 , 1.        , 1.        ,
        0.37980849, 0.        , 0.60769359],
       [1.        , 1.        , 0.46037393, 1.        , 1.        ,
        0.65054025, 1.        , 0.        ]])

### STEP 5 : Calculate Concordance & Discordance Threshold

In [393]:
def calculate_threshold(matrix):
    threshold = 0
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if (i == j):
                continue
            threshold += matrix[i][j]
            
    return threshold / (matrix.shape[0] * (matrix.shape[0] - 1))

In [394]:
concordance_threshold = calculate_threshold(concordance_matrix)
concordance_threshold

0.5232142857142855

In [395]:
discordance_threshold = calculate_threshold(discordance_matrix)
discordance_threshold

0.7766179732847552

In [396]:
concordance_matrix

array([[0.  , 0.85, 0.85, 0.7 , 1.  , 0.85, 0.7 , 0.85],
       [0.45, 0.  , 0.7 , 0.55, 0.45, 0.9 , 0.6 , 0.6 ],
       [0.15, 0.3 , 0.  , 0.25, 0.15, 0.3 , 0.6 , 0.6 ],
       [0.3 , 0.45, 0.75, 0.  , 0.45, 0.45, 0.6 , 0.75],
       [0.15, 0.55, 0.85, 0.7 , 0.  , 0.75, 0.6 , 0.85],
       [0.25, 0.4 , 0.7 , 0.55, 0.25, 0.  , 0.7 , 0.7 ],
       [0.4 , 0.4 , 0.4 , 0.1 , 0.4 , 0.4 , 0.  , 0.85],
       [0.15, 0.55, 0.4 , 0.25, 0.25, 0.45, 0.15, 0.  ]])

In [397]:
discordance_matrix

array([[0.        , 0.51792067, 0.51792067, 0.72496838, 0.        ,
        0.34528045, 0.51239464, 0.42460764],
       [1.        , 0.        , 0.51792067, 1.        , 1.        ,
        0.68968525, 1.        , 0.51239464],
       [1.        , 1.        , 0.        , 1.        , 1.        ,
        1.        , 1.        , 1.        ],
       [1.        , 0.46037393, 0.49325779, 0.        , 0.68968525,
        0.34528045, 0.94237606, 0.55244872],
       [1.        , 0.92074787, 0.62778264, 1.        , 0.        ,
        0.55244872, 0.76859195, 0.56614352],
       [1.        , 1.        , 0.6905609 , 1.        , 1.        ,
        0.        , 1.        , 1.        ],
       [1.        , 0.50641133, 0.5129881 , 1.        , 1.        ,
        0.37980849, 0.        , 0.60769359],
       [1.        , 1.        , 0.46037393, 1.        , 1.        ,
        0.65054025, 1.        , 0.        ]])

### STEP 6 : Input Concordance & Discordance to Respective Threshold

In [398]:
def calculate_preaggregate_matrix(matrix, threshold):
    shape = matrix.shape[0]
    f_matrix = np.full((shape, shape), np.nan)
        
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if (i == j):
                continue
            f_matrix[i][j] = 1 if matrix[i][j] >= threshold else 0
    
    return f_matrix

In [399]:
f_concordance = calculate_preaggregate_matrix(concordance_matrix, concordance_threshold)
f_concordance

array([[nan,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 0., nan,  1.,  1.,  0.,  1.,  1.,  1.],
       [ 0.,  0., nan,  0.,  0.,  0.,  1.,  1.],
       [ 0.,  0.,  1., nan,  0.,  0.,  1.,  1.],
       [ 0.,  1.,  1.,  1., nan,  1.,  1.,  1.],
       [ 0.,  0.,  1.,  1.,  0., nan,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0., nan,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0., nan]])

In [400]:
f_discordance = calculate_preaggregate_matrix(discordance_matrix, discordance_threshold)
f_discordance

array([[nan,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1., nan,  0.,  1.,  1.,  0.,  1.,  0.],
       [ 1.,  1., nan,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  0.,  0., nan,  0.,  0.,  1.,  0.],
       [ 1.,  1.,  0.,  1., nan,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  1.,  1., nan,  1.,  1.],
       [ 1.,  0.,  0.,  1.,  1.,  0., nan,  0.],
       [ 1.,  1.,  0.,  1.,  1.,  0.,  1., nan]])

### STEP 7 : Concordance & Discordance Aggregation to form Dominance Matrix

In [401]:
dominance_matrix = f_concordance * f_discordance
dominance_matrix

array([[nan,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., nan,  0.,  1.,  0.,  0.,  1.,  0.],
       [ 0.,  0., nan,  0.,  0.,  0.,  1.,  1.],
       [ 0.,  0.,  0., nan,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  1., nan,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0., nan,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0., nan,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0., nan]])

### STEP 8 : Rank the Alternatives

- Remove Dominated Alternatives where column have at least one cell with value == 1

In [402]:
def remove_dominated_alternatives(dominance_matrix):
    dominance_matrix = np.where(np.isnan(dominance_matrix), 0, dominance_matrix)
    dominance_matrix_binary = np.where(dominance_matrix > 0, 1, dominance_matrix)

    eliminate_columns = np.any(dominance_matrix_binary == 1, axis=0)

    retained_indices = np.where(~eliminate_columns)[0]

    dominance_matrix_cleaned = np.delete(dominance_matrix, np.where(eliminate_columns)[0], axis=1)

    return dominance_matrix_cleaned, retained_indices

In [403]:
dominant_alt_matrix, alt_indices = remove_dominated_alternatives(dominance_matrix)
print("Dominant Alternative Matrix : ", dominant_alt_matrix)
print("Alternatives : ", alt_indices)

Dominant Alternative Matrix :  [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Alternatives :  [0 2 4 5]


- For the filtered (dominant) alternatives, sort the rank according to the sum of difference between the corresponding cell of concordance and discordance matrix

In [404]:
def calculate_rank(indices, c_matrix, d_matrix):
    ranks = {}

    for i in range(len(indices)):
        score = 0
        for j in range(c_matrix.shape[0]):
            score += (c_matrix[i][j] - d_matrix[i][j])
        ranks[i] = score

    sorted_ranks = dict(sorted(ranks.items(), key=lambda item: item[1], reverse=True))

    return sorted_ranks

In [405]:
result = calculate_rank(alt_indices, concordance_matrix, discordance_matrix)

for rank, (alternative, score) in enumerate(result.items(), start=1):
    print(f"Alternative {alternative + 1} is ranked number {rank} with preference score : {score}")

Alternative 1 is ranked number 1 with preference score : 2.7569075465560617
Alternative 4 is ranked number 2 with preference score : -0.7334222039904296
Alternative 2 is ranked number 3 with preference score : -1.470000563132185
Alternative 3 is ranked number 4 with preference score : -4.65
